# CICFlows

In [1]:
import os
import pandas as pd
import datetime
import calendar
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import pickle

In [2]:
data = pd.read_csv(
    './data/IDS2017Data/netflow_data/Wednesday-workingHours.pcap_ISCX.csv')
data.shape

/home/talhajavaid32/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(692703, 85)

In [3]:
data.sample(20)

Flow ID      Source IP  \
203558        172.16.0.1-192.168.10.50-39094-80-6     172.16.0.1   
478843    192.168.10.16-54.192.36.144-43276-443-6  192.168.10.16   
425592      192.168.10.3-192.168.10.8-53-58382-17   192.168.10.8   
334672    178.172.160.3-192.168.10.15-443-55076-6  192.168.10.15   
130999        172.16.0.1-192.168.10.50-50806-80-6     172.16.0.1   
689039     192.168.10.3-192.168.10.14-53-60780-17  192.168.10.14   
570489     192.168.10.3-192.168.10.15-53-54506-17  192.168.10.15   
690749     192.168.10.3-192.168.10.51-53-18778-17  192.168.10.51   
55525        172.16.0.1-192.168.10.51-443-48498-6  192.168.10.51   
194298        172.16.0.1-192.168.10.50-54018-80-6     172.16.0.1   
533497     192.168.10.3-192.168.10.17-53-37505-17  192.168.10.17   
80614         172.16.0.1-192.168.10.50-52020-80-6     172.16.0.1   
448164   192.168.10.14-69.172.216.111-56470-443-6  192.168.10.14   
39989   188.132.227.242-192.168.10.51-443-51807-6  192.168.10.51   
478729     192.168.10.3-192.168.10.16-53-17134-17  192.168.10.16   
321845        172.16.0.1-192.168.10.50-46784-80-6     172.16.0.1   
275299        172.16.0.1-192.168.10.50-51892-80-6     172.16.0.1   
201736        172.16.0.1-192.168.10.50-37434-80-6     172.16.0.1   
532775      192.168.10.3-192.168.10.9-53-53650-17   192.168.10.9   
347893      192.168.10.1-192.168.10.3-53-61654-17   192.168.10.3   

         Source Port   Destination IP   Destination Port   Protocol  \
203558         39094    192.168.10.50                 80          6   
478843         43276    54.192.36.144                443          6   
425592         58382     192.168.10.3                 53         17   
334672         55076    178.172.160.3                443          6   
130999         50806    192.168.10.50                 80          6   
689039         60780     192.168.10.3                 53         17   
570489         54506     192.168.10.3                 53         17   
690749         18778     192.168.10.3                 53         17   
55525          48498       172.16.0.1                443          6   
194298         54018    192.168.10.50                 80          6   
533497         37505     192.168.10.3                 53         17   
80614          52020    192.168.10.50                 80          6   
448164         56470   69.172.216.111                443          6   
39989          51807  188.132.227.242                443          6   
478729         17134     192.168.10.3                 53         17   
321845         46784    192.168.10.50                 80          6   
275299         51892    192.168.10.50                 80          6   
201736         37434    192.168.10.50                 80          6   
532775         53650     192.168.10.3                 53         17   
347893         61654     192.168.10.1                 53         17   

             Timestamp   Flow Duration   Total Fwd Packets  \
203558  5/7/2017 10:52        86037124                   6   
478843   5/7/2017 3:45         5977068                  10   
425592   5/7/2017 2:15           30873                   2   
334672  5/7/2017 11:23           11079                   2   
130999  5/7/2017 10:46             971                   2   
689039   5/7/2017 4:48           72368                   4   
570489  5/7/2017 10:13           23889                   2   
690749  5/7/2017 12:38           30692                   2   
55525   5/7/2017 10:26          214813                   1   
194298  5/7/2017 10:51        99316913                   9   
533497   5/7/2017 3:22             236                   2   
80614   5/7/2017 10:43           18642                   2   
448164   5/7/2017 3:10           74077                   2   
39989   5/7/2017 10:05       116601208                  43   
478729  5/7/2017 10:03             272                   2   
321845  5/7/2017 11:00           13988                   4   
275299  5/7/2017 10:57        98922926                   8   
20173

In [4]:
data.columns = data.columns.str.lower().str.strip().str.replace(
    ' ', '_').str.replace('/s', '').str.replace('/', '_').str.replace('.', '_')

In [5]:
cols = list(data.columns)
cols.remove('fwd_header_length_1')
data = data[cols]

In [6]:
data[['flow_bytes', 'flow_packets']] = data[[
    'flow_bytes', 'flow_packets']].astype(pd.np.float64)
# Filter out HeartBleed
data = data[data.label != 'Heartbleed']

In [7]:
data.label.unique()

array(['BENIGN', 'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk',
       'DoS GoldenEye'], dtype=object)

## Convert timestamp to utc timestamp

In [8]:
# Datetime to UTC Timestamp
dt2ts = lambda x: calendar.timegm(x.timetuple())

In [9]:
# Converting datetimestamps to UTC
data['utc_timestamp'] = data.apply(lambda row: dt2ts(datetime.datetime.strptime(
    row['timestamp'], '%d/%m/%Y %H:%M')), axis=1)
data[['timestamp', 'utc_timestamp']].sample(20)

timestamp  utc_timestamp
366580  5/7/2017 12:59     1499259540
519610   5/7/2017 1:56     1499219760
494086   5/7/2017 4:20     1499228400
177833  5/7/2017 10:50     1499251800
468872  5/7/2017 11:58     1499255880
552314  5/7/2017 11:50     1499255400
651234   5/7/2017 9:50     1499248200
200777  5/7/2017 10:51     1499251860
134467  5/7/2017 10:46     1499251560
692524  5/7/2017 10:57     1499252220
220334  5/7/2017 10:53     1499251980
301818  5/7/2017 10:59     1499252340
274158  5/7/2017 10:57     1499252220
475390   5/7/2017 3:24     1499225040
420125   5/7/2017 1:49     1499219340
345786  5/7/2017 11:49     1499255340
15549    5/7/2017 9:23     1499246580
244778  5/7/2017 10:55     1499252100
684859   5/7/2017 2:11     1499220660
500988   5/7/2017 4:35     1499229300

## Time ranges of each attack

In [18]:
'''
dict of format {
    'attack_name': (
            (starttime, endtime),
            (utc_start_time, utc_end_time),
            {'attack | benign':
                {'count | unique_source_ips | unique_source_ports | 
                unique_destination_ips | unique_source_ports, unique_protocols': value}
            }
        )
    }
''' 
attack_identifiers = {}
for l in list(data.label.unique()):
    if l != 'BENIGN':
        stidx = data[data.label == l].utc_timestamp.idxmin()
        etidx = data[data.label == l].utc_timestamp.idxmax()
        timetup = (data.loc[stidx, 'timestamp'], data.loc[etidx, 'timestamp'])
        stutctime = data.loc[stidx, 'utc_timestamp']
        etutctime = data.loc[etidx, 'utc_timestamp']
        utctimetup = (stutctime, etutctime)
        attack_dict = {}
        benign_dict = {}
        
        rangemask = ((data.utc_timestamp >= stutctime) &
                     (data.utc_timestamp <= etutctime))
        attackmask = (rangemask & (data.label == l))
        benignmask = (rangemask & (data.label == 'BENIGN'))
        attackcount = attackmask.sum()
        benigncount = benignmask.sum()
        attackflows = data.loc[attackmask]
        benignflows = data.loc[benignmask]
        
        attack_dict['count'] = attackcount
        attack_dict['unique_source_ips'] = attackflows['source_ip'].unique()
        attack_dict['unique_source_ports'] = attackflows['source_port'].unique()
        attack_dict['unique_destination_ips'] = attackflows['destination_ip'].unique()
        attack_dict['unique_destination_ports'] = attackflows['destination_port'].unique()
        
        benign_dict['count'] = benigncount
        benign_dict['unique_source_ips'] = benignflows['source_ip'].unique()
        benign_dict['unique_source_ports'] = benignflows['source_port'].unique()
        benign_dict['unique_destination_ips'] = benignflows['destination_ip'].unique()
        benign_dict['unique_destination_ports'] = benignflows['destination_port'].unique()
        
        attack_identifiers[l] = (timetup, utctimetup, {'attack': attack_dict, 'benign': benign_dict}) 
        
#         print l
#         print 'time range:', data.loc[stidx, 'timestamp'], '-', data.loc[etidx, 'timestamp']
#         print 'utc time range:', stutctime, etutctime
#         print 'Attack count:', attackcount, 'Benign count:', benigncount
#         print 'Attack'
#         print 'Unique Source IPs', attackflows['source_ip'].unique()
#         print 'Unique Source ports', attackflows['source_port'].unique() 
#         print 'Unique Destination IPs', attackflows['destination_ip'].unique()
#         print 'Unique Destination ports', attackflows['destination_port'].unique()
        
#         print 'Benign'
#         print 'Unique Source IPs', benignflows['source_ip'].unique() 
#         print 'Unique Source ports', benignflows['source_port'].unique() 
#         print 'Unique Destination IPs', benignflows['destination_ip'].unique()
#         print 'Unique Destination ports', benignflows['destination_port'].unique()
#         print '\n'
print attack_identifiers

{'DoS slowloris': (('5/7/2017 2:24', '5/7/2017 10:11'), (1499221440, 1499249460), {'benign': {'count': 232979, 'unique_source_ports': array([49459, 49453, 46124, ...,  5776,  4260, 29671]), 'unique_destination_ports': array([   80,   389,    88, ..., 45294, 51621, 35450]), 'unique_source_ips': array(['192.168.10.14', '192.168.10.17', '192.168.10.16', ...,
       '54.192.37.75', '54.210.65.106', '54.165.169.242'], dtype=object), 'unique_destination_ips': array(['209.48.71.168', '192.168.10.3', '91.189.88.161', ...,
       '195.93.247.64', '88.85.71.60', '17.253.20.125'], dtype=object)}, 'attack': {'count': 5796, 'unique_source_ports': array([49631, 49632, 49633, ..., 33358, 33360, 33362]), 'unique_destination_ports': array([80]), 'unique_source_ips': array(['172.16.0.1'], dtype=object), 'unique_destination_ips': array(['192.168.10.50'], dtype=object)}}), 'DoS Hulk': (('5/7/2017 10:43', '5/7/2017 11:07'), (1499251380, 1499252820), {'benign': {'count': 27532, 'unique_source_ports': array(

In [29]:
for k, v in attack_identifiers.iteritems():
    print k
    rangemask = ((data.utc_timestamp >= v[1][0]) & (data.utc_timestamp <= v[1][1]))
    print (rangemask & (data.source_ip == v[2]['attack']['unique_source_ips'][0]) & (data.destination_ip == v[2]['attack']['unique_destination_ips'][0]) & (data.label == 'BENIGN')).sum() 

DoS slowloris
0
DoS Hulk
0
DoS Slowhttptest
0
DoS GoldenEye
0


In [30]:
with open('./attack_identifiers.p', 'wb') as p:
    pickle.dump(attack_identifiers, p)

# Intraflow

In [1]:
import os
import pandas as pd
import datetime
import calendar
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import pickle
import time

In [2]:
data = pd.read_csv(
    './data/IDS2017Data/DDoS-WednesdayData/intraflow_data/Wednesday-workingHours.csv')
data.shape

(615250, 668)

In [3]:
data.sample(20)

key             sa              da  \
601959     19216810141921681035706453378  192.168.10.14    192.168.10.3   
466611      1921681091921681035604753196   192.168.10.9    192.168.10.3   
13801     192168108104161652284943444392   192.168.10.8  104.16.165.228   
480749        19216810852212787570484434   192.168.10.8     52.21.27.87   
452153      1921681031921681016042753225   192.168.10.3    192.168.10.1   
535043      1921681051921681035197453246   192.168.10.5    192.168.10.3   
80816         19216810162350752748470800  192.168.10.16     23.50.75.27   
568950      1921681031921681016174253164   192.168.10.3    192.168.10.1   
450394   1921681014130211421615499244392  192.168.10.14  130.211.42.161   
600341        19216810197221912937499800  192.168.10.19     72.21.91.29   
89241        172160119216810505392480483     172.16.0.1   192.168.10.50   
612542       192168101269311165954998800  192.168.10.12    69.31.116.59   
2787        1921681092319920625413936804   192.168.10.9  23.199.206.254   
613643  19216810162319414147603224436497  192.168.10.16   23.194.141.47   
73518      19216810171921681034116453244  192.168.10.17    192.168.10.3   
304588     17216011921681050466748011918     172.16.0.1   192.168.10.50   
537000  19216810516220820178604444434134   192.168.10.5  162.208.20.178   
511721    192168101423194101189561064433  192.168.10.14  23.194.101.189   
228870     17216011921681050546028012009     172.16.0.1   192.168.10.50   
175705     17216011921681050399308011926     172.16.0.1   192.168.10.50   

                  ts            te     dp       sp  ip  op     ib    ...     \
601959  1.499284e+09  1.499284e+09   53.0  57064.0   2   4    270    ...      
466611  1.499275e+09  1.499275e+09   53.0  56047.0   2   2    114    ...      
13801   1.499256e+09  1.499256e+09  443.0  49434.0   1   2     46    ...      
480749  1.499276e+09  1.499276e+09  443.0  57048.0   3   4      0    ...      
452153  1.499275e+09  1.499275e+09   53.0  60427.0   1   1    176    ...      
535043  1.499279e+09  1.499279e+09   53.0  51974.0   2   2    162    ...      
80816   1.499260e+09  1.499260e+09   80.0  48470.0   4   4      0    ...      
568950  1.499282e+09  1.499282e+09   53.0  61742.0   1   1    120    ...      
450394  1.499274e+09  1.499274e+09  443.0  54992.0   2   2     46    ...      
600341  1.499284e+09  1.499284e+09   80.0  37499.0   3   3      0    ...      
89241   1.499260e+09  1.499260e+09   80.0  53924.0   2   2    483    ...      
612542  1.499285e+09  1.499285e+09   80.0  54998.0   4   4      0    ...      
2787    1.499255e+09  1.499255e+09   80.0  13936.0   4   4      0    ...      
613643  1.499285e+09  1.499285e+09  443.0  60322.0   9  12   5840    ...      
73518   1.499260e+09  1.499260e+09   53.0  41164.0   2   2    172    ...      
304588  1.499263e+09  1.499263e+09   80.0  46674.0   5   8  11595    ...      
537000  1.499279e+09  1.499279e+09  443.0  60444.0   8   8   3776    ...      
511721  1.499278e+09  1.499278e+09  443.0  56106.0   3   3      0    ...      
228870  1.499263e+09  1.499263e+09   80.0  54602.0   7   7  11595    ...      
175705  1.499262e+09  1.499262e+09   80.0  39930.0   6   8  11595    ...      

        fipt_91  fipt_92  fipt_93  fipt_94  fipt_95  fipt_96  fipt_97  \
601959      1.0      0.0      0.0      0.0      0.0      0.0      0.0   
466611      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
13801       0.0      0.0      0.0      0.0      0.0      0.0      0.0   
480749      0.0      1.0      0.0      0.0      0.0      0.0      0.0   
452153      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
535043      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
80816       1.0      0.0      0.0      0.0      0.0      0.0      0.0   
568950      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
450394      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
600341      1.0      0.0      0.0      0.0      0.0      0.0      0.0  

In [4]:
utctimetup = datetime.datetime.utcfromtimestamp(0)
print str(utctimetup)

# time.gmtime(0)

1970-01-01 00:00:00


In [4]:
with open('./attack_identifiers.p', 'rb') as p:
    attack_identifiers = pickle.load(p)

In [11]:
print data.ts.min(), data.te.max()
for k, v in attack_identifiers.iteritems():
    print k
    print v[1][0], v[1][1]
#     print rangemask.sum()
#     print (rangemask & (data.sa == v[2]['attack']['unique_source_ips'][0]) & (data.da == v[2]['attack']['unique_destination_ips'][0])).sum(), v[2]['attack']['count'] 

1499254962.084372 1499285414.560806
DoS slowloris
1499221440 1499249460
DoS Hulk
1499251380 1499252820
DoS Slowhttptest
1499249700 1499251020
DoS GoldenEye
1499253000 1499253540
